# 디스크립터 분석

## 파이썬 내부에서의 디스크립터 활용
좋은 디스크립터인지 확인하는 간단한 방법은 다른 파이썬 객체와 얼마나 유사한 지를 보는 것  
좋은 디스크립터는 파이썬 자체의 디스크립터와 동일  
파이썬이 내부 로직의 일부를 해결하기 위해 디스크립터를 사용하는 가장 일반적인 시나리오 살펴보기  

### 함수와 메서드
함수는 &#95;&#95;get&#95;&#95; 메서드를 구현했기 때문에 클래스 안에서 메서드처럼 동작 가능  
메서드 = 추가 파라미터를 가진 함수.
- 관습적으로 메서드의 첫 번째 파라미터는 "self"라는 이름을 사용하여 메서드를 소유하고 있는 클래스의 인스턴스를 나타냄  
- 메서드에서 "self"를 사용하는 것은 객체를 받아서 수정을 하는 함수를 사용하는 것과 동일  
```python
class MyClass:
    def method(self, ...):
        self.x = 1
```
위의 코드는 아래처럼 정의하는 것과 같음
```python
class MyClass: 
    pass
def method(myclass_instance, ...):
    myclass_instance.x = 1
method(MyClass())
```

따라서 메서드는 객체를 수정하는 또 다른 함수일 뿐이며, 객체 안에서 정의되었기 때문에 객체에 바인딩되어 있다고 말함
```python
instance = MyClass()
instance.method(...)
```
위와 같이 호출하면 파이썬은 실제로 아래와 같이 처리함
```python
instance = MyClass()
MyClass.method(instance, ...)
```

함수는 디스크립터 프로토콜을 구현했으므로 메서드를 호출하기 전에 &#95;&#95;get&#95;&#95;() 메서드가 먼저 호출되고 필요한 변환을 함 

In [1]:
def function(): pass

function.__get__

<method-wrapper '__get__' of function object at 0x000001B1165A5DC0>

instance.method(...) 구문에서는 괄호 안의 인자를 처리하기 전에 "instance.method" 부분이 먼저 평가됨  
method는 클래스 속성으로 정의된 객체이고 &#95;&#95;get&#95;&#95;메서드가 있기 때문에 &#95;&#95;get&#95;&#95;메서드가 호출됨  
&#95;&#95;get&#95;&#95; 메서드가 하는 일 = 함수를 메서드로 변환하는 것 = 함수를 작업하려는 객체의 인스턴스에 바인딩  

아래 예제는 외부에서 호출 가능한 형태의 함수 or 메서드를 클래스 내에 호출 가능한 객체로 정의한 것
- Method 클래스의 인스턴스는 함수나 메서드 형태로 다른 클래스에서 사용될 것  
- 이 함수는 단지 전달받은 3개의 인자를 그대로 출력  
- 첫 번째 파라미터: instance. 클래스에 정의될 경우 self 가 됨
- &#95;&#95;call&#95;&#95;() 메서드에서 self는 MyClass의 인스턴스가 아닌 Method의 인스턴스를 나타내는 것. (파라미터로 전달된 instance가 MyClass 타입의 객체임)

In [2]:
class Method:
    def __init__(self, name):
        self.name = name
        
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출됨. 인자는 {arg1}와 {arg2}입니다.")
        
class MyClass:
    method = Method("Internal call")

In [5]:
# case1
instance = MyClass()
Method("External call")(instance, "first", "second")

External call: <__main__.MyClass object at 0x000001B11769B160> 호출됨. 인자는 first와 second입니다.


In [6]:
# case2
instance.method("first", "second")

TypeError: __call__() missing 1 required positional argument: 'arg2'

case2에서 오류가 발생한 이유는 파라미터의 위치가 한 칸씩 밀려서 Method.&#95;&#95;call&#95;&#95;기준으로 self자리에 instance가 전달되고, instance 자리에 "first"가 전달되고 arg1 자리에 "second"가 전달되는데, arg2 자리에는 아무 값도 전달되지 않았기 때문  

이 문제를 해결하려면 메서드를 디스크립터로 변경하면 됨  
디스크립터로 변경하면 instance.method 호출 시 Method.&#95;&#95;get&#95;&#95; 메서드를 먼저 호출할 것임.  
여기에서 첫 번째 파라미터로 Method의 인스턴스를 전달함으로써 객체에 바인딩하면 됨

In [13]:
from types import MethodType

class Method:
    def __init__(self, name):
        self.name = name
        
    def __call__(self, instance, arg1, arg2):
        print(f"{self.name}: {instance} 호출됨. 인자는 {arg1}와 {arg2}입니다.")
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return MethodType(self, instance)
    
class MyClass:
    method = Method("Internal call")

In [14]:
# case1
instance = MyClass()
Method("External call")(instance, "first", "second")

External call: <__main__.MyClass object at 0x000001B1176BB0D0> 호출됨. 인자는 first와 second입니다.


In [15]:
# case2
instance.method("first", "second")

Internal call: <__main__.MyClass object at 0x000001B1176BB0D0> 호출됨. 인자는 first와 second입니다.


이제 두가지 케이스의 호출 모두 성공함  

수정한 사항은 types 모듈의 MethodType을 사용하여 함수를 메서드로 변환하는 것.  
이 클래스의 첫 번째 파라미터는 호출 가능한 것이어야 하는데, 여기에서는 self로 self는 Method의 인스턴스로 &#95;&#95;call&#95;&#95;메서드를 구현했으므로 호출 가능한 형태임  
두번째 파라미터는 이 함수에 바인딩 할 객체임  

파이썬의 함수 객체도 이와 비슷하게 동작하므로 클래스 내부에 함수를 정의할 경우 메서드처럼 사용 가능한 것  
사용자 정의 호출 가능한 객체를 정의할 때는 지금처럼 디스크립터로 만들어서 클래스 속성으로도 사용할 수 있도록 하는 것이 좋음

### 메서드를 위한 빌트인 데코레이터  
@property, @classmethod, @staticmethod 데코레이터는 디스크립터  

메서드를 인스턴스가 아닌 클래스에서 직접 호출할 때는 관습적으로 디스크립터 자체를 반환  
프로퍼티를 클래스에서 직접 호출하면 계산할 속성이 없으므로 일종의 디스크립터인 프로퍼티 객체 자체를 반환함  

- @classmethod를 사용하면 디스크립터의 &#95;&#95;get&#95;&#95; 함수가 메서드를 인스턴스에서 호출하든 클래스에서 직접 호출하든 상관 없이 데코레이팅 함수에 첫 번째 파라미터로 메서드를 소유한 클래스를 넘겨줌  
- @staticmethod를 사용하면 정의한 파라미터 이외의 파라미터를 넘기지 않도록 함. 
    - &#95;&#95;get&#95;&#95;메서드에서 함수의 첫 번째 파라미터에 self를 바인딩하는 작업을 취소
    
예를 들어 @property 데코레이터처럼 동작하나 클래스를 대상으로 한다는 것만 다른 @classproperty 데코레이터를 만들었다고 가정하면 아래의 코드가 정상 동작해야함
```python
class TableEvent:
    schema = "public"
    table = "user"
    
    @classproperty
    def topic(cls):
        prefix = read_prefix_from_config()
        return f"{prefix}{cls.schema}.{cls.table}"
```
> TableEvent.topic  
결과 >> 'public.user'

> TableEvent().topic  
결과 >> 'public.user'

### 슬롯(slot)
클래스에 &#95;&#95;slot&#95;&#95; 속성을 정의하면 클래스가 기대하는 특정 속성만 정의하고 다른 것은 제한할 수 있음  
&#95;&#95;slot&#95;&#95;에 정의되지 않은 속성을 동적으로 추가하려고 할 경우 AttributeError가 발생  
이 속성을 정의하면 클래스는 정적으로 되고, &#95;&#95;dict&#95;&#95; 속성을 갖지 않으므로 객체에 동적으로 속성을 추가할 수 없음  

**객체의 사전이 없는데 속성을 가져올 수 있는 이유**: 디스크립터 사용  
- &#95;&#95;slot&#95;&#95;에 정의된 이름마다 디스크립터를 만들어서 값을 저장하고 있으므로 나중에 검색도 가능

In [19]:
class Coordinate2D:
    __slots__ = ("lat", "long")
    
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.lat}, {self.long})"

위의 기능은 파이썬의 동적인 특성을 없애므로 조심해서 사용할 필요가 있음  
일반적으로 이는 정적인 객체에만 사용해야 하며 코드의 다른 부분에서 절대로 동적으로 속성을 추가할 일이 없다는 것을 확신할 때에만 사용할 것  
슬롯을 사용한 객체의 장점 = 메모리를 덜 사용함 => 사전 형태가 아닌 고정된 필드의 값만 저장하면 되기 때문

## 데코레이터를 디스크립터로 구현하기  
위에서 파이썬이 디스크립터를 사용하여 함수를 클래스 안에 선언했을 때도 메서드처럼 사용할 수 있게 하는 것을 살펴봄  
데코레이터가 호출되는 형태에 맞춰 동작하도록 디스크립터 프로토콜의 &#95;&#95;get&#95;&#95; 메서드를 활용하는 방법도 알아봄  
위의 기법을 활용하면 사용자 정의 데코레이터 개발 중 발생하는 문제 해결 가능  

데코레이터를 이와 같은 형태로 만들기 위한 일반적인 방법은 &#95;&#95;get&#95;&#95; 메서드를 구현하고 types.MethodType을 사용해 데코레이터 자체를 객체에 바인딩된 메서드로 만드는 것  
이렇게 하려면 데코레이터를 객체로 구현해야 하는데 만약 함수로 구현하는 경우 &#95;&#95;get&#95;&#95;() 메서드가 이미 존재할 것이기 때문에 정상적으로 동작하지 않게 됨.  
따라서 더 깔끔한 방법은 데코레이터를 위한 클래스를 정의하는 것